##### 本次作业的问题
1、数据的获取需要改进：

a.> 地铁数据的获取，爬取代码不具备通用性，不同地铁线路的页面布局可能有差异，标签提取的时候写了很多if-else。有没有更好的爬虫思路？

2、本次练习新学到的知识点记录：

a.> 使用beautifulsoup进行标签定位时，遇到\<br\>这样的标签导致的问题。

解决方法：https://blog.csdn.net/u012587107/article/details/80543977

b.> 同时提取多个不同的标签，可以使用正则表达式。




In [ ]:
##########################地铁路线规划问题
#######地铁数据获取
import requests
from bs4 import BeautifulSoup
import re
import string
from urllib.parse import quote

##伪装为浏览器
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',}
url = 'https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485'##北京地铁图
response = requests.get(url,headers=headers)
response = response.content.decode("UTF-8")

##爬取地铁线的百度百科网址,获取地铁线路和对应url
bs = BeautifulSoup(response,"html.parser")
urls_html = bs.find("caption",text="北京地铁开通简表").parent

urls_dict = {}
pattern = re.compile('<a href="(/item/.+?)" target="_blank">([\w\u4e00-\u9fa5]+线)</a>')
for (url,name) in re.findall(pattern,str(urls_html)):
    urls_dict[name]= "https://baike.baidu.com" + url
list(urls_dict)
urls_dict

In [34]:
####根据urls_dict中的地铁线路爬取地铁站
lines_stations = {}
for (name,url) in list(urls_dict.items()):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',}
    response = requests.get(url,headers=headers)
    response = response.content.decode("UTF-8").replace("br","").replace("br/","")##先去除掉标签<br>，不然下面使用bs提取标签文本的时候，有时候会返回None
    bs = BeautifulSoup(response,"html.parser")
    
    if bs.find("b",text=re.compile(".*首末.*车[(时刻时间)]+.*")) is  not None:
        table_html = bs.find("b",text=re.compile(".*首末.*车[(时刻时间)]+.*")).parent.parent.parent
        stations = []
        for i in range(len(table_html.findAll("tr"))):
            s = table_html.findAll("tr")[i].find(re.compile("(td)|(th)")).get_text()
            if len(s)<9 and s not in ["车站名称","首车时间","备注"]:
                stations.append(s)
        print(name,stations)
        lines_stations[name]=stations
    elif bs.find("caption",text=re.compile(".*首末.*车[(时刻时间)]+.*")) is not None:
        table_html = bs.find("caption",text=re.compile(".*首末.*车[(时刻时间)]+.*")).parent
        stations = []
        for i in range(len(table_html.findAll("tr"))):
            s = table_html.findAll("tr")[i].find(re.compile("(td)|(th)")).get_text()
            if len(s)<9 and s not in ["车站名称","首车时间","备注"]:
                stations.append(s)
        print(name,stations)
        lines_stations[name]=stations
    elif bs.find("td",text=re.compile(".*首末.*车[(时刻时间)]+.*")) is not None:
        table_html = bs.find("td",text=re.compile(".*首末.*车[(时刻时间)]+.*")).parent.parent
        stations = []
        for i in range(len(table_html.findAll("tr"))):
            s = table_html.findAll("tr")[i].find(re.compile("(td)|(th)")).get_text()
            if len(s)<9 and s not in ["车站名称","首车时间","备注"]:
                stations.append(s)
        print(name,stations)
        lines_stations[name]=stations
    elif bs.find("caption",text=re.compile("车站信息.*")) is not None:
        table_html = bs.find("caption",text=re.compile("车站信息.*")).parent
        stations = []
        for i in range(len(table_html.findAll("tr"))):
            s = table_html.findAll("tr")[i].findAll(re.compile("(th)|(td)"))[1].get_text().replace("\n","")
            if len(s)<9 and s not in ["车站名称","首车时间"]:
                stations.append(s)
        print(name,stations)
        lines_stations[name]=stations
    elif bs.find("th",text=re.compile("车站名称")) is not None:
        table_html = bs.find("th",text=re.compile("车站名称")).parent.parent
        stations = []
        for i in range(len(table_html.findAll("tr"))):
            s = table_html.findAll("tr")[i].find(re.compile("(div)|(th)"))
            if s is not None:
                if len(s.get_text())<9 and s.get_text() != "车站名称": 
                    stations.append(s.get_text())
        print(name,stations)
        lines_stations[name]=stations
    elif bs.find("div",text=re.compile("车站名称")) is not None:
        table_html = bs.find("div",text=re.compile("车站名称")).parent.parent.parent
        stations = []
        for i in range(len(table_html.findAll("tr"))):
            s = table_html.findAll("tr")[i].find("div")
            if s is not None:
                if len(s.get_text())<9 and s.get_text() != "车站名称":            
                    stations.append(s.get_text())
        print(name,stations)
        lines_stations[name]=stations
    else:
        print(name,url)



北京地铁1号线 ['苹果园', '古城', '八角游乐园', '八宝山', '玉泉路', '五棵松', '万寿路', '公主坟', '军事博物馆', '木樨地', '南礼士路', '复兴门', '西单', '天安门西', '天安门东', '王府井', '东单', '建国门', '永安里', '国贸', '大望路', '四惠', '四惠东']
复八线 ['苹果园', '古城', '八角游乐园', '八宝山', '玉泉路', '五棵松', '万寿路', '公主坟', '军事博物馆', '木樨地', '南礼士路', '复兴门', '西单', '天安门西', '天安门东', '王府井', '东单', '建国门', '永安里', '国贸', '大望路', '四惠', '四惠东']
北京地铁13号线 ['西直门', '明光桥西', '大钟寺', '知春路', '五道口', '清华东路西口', '上地', '清河站', '西二旗', '龙泽', '回龙观', '霍营', '建材城东路', '立水桥', '北苑', '来广营西', '望京西', '望和桥', '芍药居', '光熙门', '柳芳', '东直门']
北京地铁八通线 ['四惠', '四惠东', '高碑店', '传媒大学', '双桥', '管庄', '八里桥', '通州北苑', '果园', '九棵树', '梨园', '临河里', '土桥']
北京地铁5号线 ['宋家庄', '刘家窑', '蒲黄榆', '天坛东门', '磁器口', '崇文门', '东单', '灯市口', '东四', '张自忠路', '北新桥', '雍和宫', '和平里北街', '和平西桥', '惠新西街南口', '惠新西街北口', '大屯路东', '北苑路北', '立水桥南', '立水桥', '天通苑南', '天通苑', '天通苑北']
北京地铁8号线 ['朱辛庄', '育知路', '平西府', '回龙观东大街', '霍营', '育新', '西小口', '永泰庄', '林萃桥', '森林公园南门', '奥林匹克公园', '奥体中心', '北土城', '安华桥', '安德里北街', '鼓楼大街', '什刹海', '南锣鼓巷', '中国美术馆']
北京地铁10号线 ['全程', '巴沟', '苏州街', '海淀黄庄', '知春里', '知春路', '西土城', 

In [33]:
############地铁站点数据预处理
##去掉空值,去掉换行符号，统一站名，在每个站后面添加站字
for (line,stations)in lines_stations.items():
    stations = [s.replace(r"\n","") for s in stations  if s !="" and s not in ("1161","全程","编号" )]
    stations = [s+"站" if not re.findall(re.compile(".*站"),s) else s for s in stations]
    print(line,stations)
    lines_stations[line]=stations
print(lines_stations)
    

北京地铁燕房线 ['苹果园站', '古城站', '八角游乐园站', '八宝山站', '玉泉路站', '五棵松站', '万寿路站', '公主坟站', '军事博物馆站', '木樨地站', '南礼士路站', '复兴门站', '西单站', '天安门西站', '天安门东站', '王府井站', '东单站', '建国门站', '永安里站', '国贸站', '大望路站', '四惠站', '四惠东站']
北京地铁燕房线 ['苹果园站', '古城站', '八角游乐园站', '八宝山站', '玉泉路站', '五棵松站', '万寿路站', '公主坟站', '军事博物馆站', '木樨地站', '南礼士路站', '复兴门站', '西单站', '天安门西站', '天安门东站', '王府井站', '东单站', '建国门站', '永安里站', '国贸站', '大望路站', '四惠站', '四惠东站']
北京地铁燕房线 ['西直门站', '明光桥西站', '大钟寺站', '知春路站', '五道口站', '清华东路西口站', '上地站', '清河站', '西二旗站', '龙泽站', '回龙观站', '霍营站', '建材城东路站', '立水桥站', '北苑站', '来广营西站', '望京西站', '望和桥站', '芍药居站', '光熙门站', '柳芳站', '东直门站']
北京地铁燕房线 ['四惠站', '四惠东站', '高碑店站', '传媒大学站', '双桥站', '管庄站', '八里桥站', '通州北苑站', '果园站', '九棵树站', '梨园站', '临河里站', '土桥站']
北京地铁燕房线 ['宋家庄站', '刘家窑站', '蒲黄榆站', '天坛东门站', '磁器口站', '崇文门站', '东单站', '灯市口站', '东四站', '张自忠路站', '北新桥站', '雍和宫站', '和平里北街站', '和平西桥站', '惠新西街南口站', '惠新西街北口站', '大屯路东站', '北苑路北站', '立水桥南站', '立水桥站', '天通苑南站', '天通苑站', '天通苑北站']
北京地铁燕房线 ['朱辛庄站', '育知路站', '平西府站', '回龙观东大街站', '霍营站', '育新站', '西小口站', '永泰庄站', '林萃桥站', '森林公园南门站', '奥林匹克公园站', '奥体中